In [1]:
# Create a dataframe from a sample CSV file
import pandas as pd
df = pd.read_csv('./data/dsg/timeseriesProfile.csv', parse_dates=['time'])
df.head()

,time,lon,lat,depth,station,humidity,temperature
0,1990-01-01 00:00:00,-76.5,37.5,0.0,Station1,89.708794,15.698009
1,1990-01-01 00:00:00,-76.5,37.5,10.0,Station1,55.789471,10.916656
2,1990-01-01 00:00:00,-76.5,37.5,20.0,Station1,50.176994,15.666663
3,1990-01-01 00:00:00,-76.5,37.5,30.0,Station1,36.855045,1.158752
4,1990-01-01 01:00:00,-76.5,37.5,0.0,Station1,65.016937,31.059647


In [2]:
# Save dataframe as a netCDF file
import os
import tempfile
from pocean.dsg import OrthogonalMultidimensionalTimeseriesProfile as omtsp

output_fp, output = tempfile.mkstemp()
os.close(output_fp)

axes = {
    't': 'time',
    'x': 'lon',
    'y': 'lat',
    'z': 'depth'
}
ncd = omtsp.from_dataframe(
    df.reset_index(),
    output=output,
    axes=axes,
    mode='a'
)
ncd.naming_authority = 'ioos'
ncd.id = 'Station1'
print(ncd)
ncd.close()

<class 'pocean.dsg.timeseriesProfile.om.OrthogonalMultidimensionalTimeseriesProfile'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2017-11-03T18:08:00Z
    featureType: timeSeriesProfile
    cdm_data_type: TimeseriesProfile
    naming_authority: ioos
    id: Station1
    dimensions(sizes): station(1), time(100), depth(4)
    variables(dimensions): <class 'str'> station(station), float64 lat(station), float64 lon(station), int32 crs(), float64 time(time), float64 depth(depth), int32 index(time,depth,station), float64 humidity(time,depth,station), float64 temperature(time,depth,station)
    groups: 



In [3]:
# Create an archive for the file
import shutil
import bagit
import netCDF4 as nc4

with nc4.Dataset(output) as ncd:
    urn = 'urn:ioos:station:{naming_authority}:{id}'.format(**ncd.__dict__)
    
temp_bagit_folder = tempfile.mkdtemp()
temp_data_folder = os.path.join(temp_bagit_folder, 'data')

# Create a new bag
bag = bagit.make_bag(
    temp_bagit_folder,
    checksum=['sha256']
)

# Copy netCDF file into the bag
shutil.copy2(output, temp_data_folder + '/parameter1.nc')

# Set bagit metadata
bag_meta = {
    'Bag-Count': '1 of 1',
    'Bag-Group-Identifier': 'ioos_bagit_testing',
    'Contact-Name': 'Kyle Wilcox',
    'Contact-Phone': '907-230-0304',
    'Contact-Email': 'axiom+ncei@axiomdatascience.com',
    'External-Identifier': urn,
    'External-Description':
        'Sensor data from station {}'.format(urn),
    'Internal-Sender-Identifier': urn,
    'Internal-Sender-Description':
        'Station - URN:{}'.format(urn),
    'Organization-address':
        '1016 W 6th Ave, Ste. 105, Anchorage, AK 99501, USA',
    'Source-Organization': 'Axiom Data Science',
}
# Update existing bag
bag.info.update(bag_meta)
bag.save(manifests=True, processes=4)

In [4]:
!tree $temp_bagit_folder
!cat $temp_bagit_folder/manifest-sha256.txt

/tmp/tmp7n_n1o50
├── bag-info.txt
├── bagit.txt
├── data
│   └── parameter1.nc
├── manifest-sha256.txt
└── tagmanifest-sha256.txt

1 directory, 5 files
e146e4d1850264e0bd71efb6ffb187e6f2069b564aef2c8317035bc25f28a1f1  data/parameter1.nc


In [5]:
# Copy a few more files into the bag
shutil.copy2(output, temp_data_folder + '/parameter2.nc')
shutil.copy2(output, temp_data_folder + '/parameter3.nc')
shutil.copy2(output, temp_data_folder + '/parameter4.nc')
bag.save(manifests=True, processes=4)

In [6]:
!tree $temp_bagit_folder
!cat $temp_bagit_folder/manifest-sha256.txt

/tmp/tmp7n_n1o50
├── bag-info.txt
├── bagit.txt
├── data
│   ├── parameter1.nc
│   ├── parameter2.nc
│   ├── parameter3.nc
│   └── parameter4.nc
├── manifest-sha256.txt
└── tagmanifest-sha256.txt

1 directory, 8 files
e146e4d1850264e0bd71efb6ffb187e6f2069b564aef2c8317035bc25f28a1f1  data/parameter1.nc
e146e4d1850264e0bd71efb6ffb187e6f2069b564aef2c8317035bc25f28a1f1  data/parameter2.nc
e146e4d1850264e0bd71efb6ffb187e6f2069b564aef2c8317035bc25f28a1f1  data/parameter3.nc
e146e4d1850264e0bd71efb6ffb187e6f2069b564aef2c8317035bc25f28a1f1  data/parameter4.nc
